In [ ]:
import tweepy
from IPython.display import display, Image  # This line lets you display images. We'll use that in a bit.
import requests
import json
from twitter_authentication import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True)

# Exercise 1
# Write a function that takes in the width and height and prints an image
def Display_image(width, height):
    page = requests.get('http://placekitten.com/' + width + "/" + height)
    display(Image(page.content))

In [ ]:
# Exercise 2
# Can you write a loop to show several images?
limit = 0
while limit < 3:
    Display_image(input("enter a width> "), input("enter a height> "))
    limit += 1
if limit == 3:
    print("That's all she wrote!")

In [ ]:
# Exercise 3
#Use the .json() function to get the response converted to a dictionary or list
time = requests.get("http://worldtimeapi.org/api/timezone/America/Indiana/Indianapolis")
print(time.content)

In [ ]:
# Exercise 4
# Get the time for your time zone
time = requests.get("http://worldtimeapi.org/api/timezone/America/Indiana/Indianapolis")
time.json()['datetime']

In [ ]:
# Exercise 5
# Get the time for your IP address
time = requests.get("http://worldtimeapi.org/api/ip")
time.json()['datetime']

In [ ]:
# Exercise 6
endpt = 'https://en.wikipedia.org/w/api.php'

def get_last_pages_changed(n):
    params = {'action': 'query',
          'format': 'json',
          'list': 'recentchanges',
          'rcnamespace': '0',
          'rclimit': n}
    r = requests.get(endpt, params)
    print(r.json())
    return [r['title'] for r in r.json()['query']['recentchanges']]
get_last_pages_changed(5)

In [ ]:
# Exercise 7 Use the streaming API to produce a list of 1000 tweets about a topic.
alexa_list = []
for tweet in tweepy.Cursor(api.search, 'alexa').items(2):
    alexa_list.append(tweet.created_at)
    

In [ ]:
# Exercise 8 From that list of 1000 tweets, eliminate retweets.
original_tweets = []
for tweet in tweepy.Cursor(api.search, 'alexa').items(1000):
    if 'retweeted_status' in tweet._json:
        continue
    else:
        print(tweet.text, '\n')       
        

In [ ]:
# Exercise 9 For each original tweet, create a dictionary with the number of times you see it retweeted in your dataset.
alexa_retweets = {}
for tweet in tweepy.Cursor(api.search, 'alexa').items(1000):
    if 'retweeted_status' in tweet._json:
        new_tweet = tweet.retweeted_status
        if new_tweet.id not in alexa_retweets:
            alexa_retweets[tweet.text] = 1
        else:
            alexa_retweets[tweet.text] += 1
            

In [ ]:
with open('alexa_data.json', 'w') as f:
    json.dump(alexa_list, f)

In [ ]:
# Exercise 10 Get a list of the URLs in your dataset
for tweet in tweepy.Cursor(api.search, 'alexa').items(100):
    urls = tweet.entities['urls']
    for url in urls:
        print(url['expanded_url'])

    

In [ ]:
# Exercise 11 Now, see if you can figure out how to eliminate retweets in the query instead.
alexa_list = []
for tweet in tweepy.Cursor(api.search, 'alexa -filter:retweets').items(10):
    alexa_list.append(tweet.text)
alexa_list
            


In [ ]:
# Exercise 12 Get the last 50 tweets from West Lafayette, using the search API.
for tweet in tweepy.Cursor(api.search, 'geocode:40.425869,-86.908066,100km').items(50):
    print("This tweet was made at", tweet.created_at, '\n', tweet.text, '\n')


In [ ]:
# Exercise 13 Alter the streaming algorithm to include a "locations" filter to get tweets from New York City. You need to use the order sw_lng, sw_lat, ne_lng, ne_lat for the four coordinates instead of a radius as in the search API.
class StreamListener(tweepy.StreamListener):
    def on_status(self, tweet):
        print(tweet.author.screen_name + "\t" + str(tweet.text) + "\n")
        
    def on_error(self, status_code):
        print( 'Error: ' + repr(status_code))
        return False

l = StreamListener()
streamer = tweepy.Stream(auth=auth, listener=l, tweet_mode='extended')

keywords = ['Apple', 'apple']
streamer.filter(locations = [-87,40,-86,41])


In [ ]:
# Bonus Exercise 1
follower_list = []
follower_of_followers = []
user = api.get_user('Nathananael_J')

for follower in user.followers(count=20):
    follower_list.append(follower.screen_name)
    
for name in follower_list:
    followers_of_friends = api.get_user(name)
    for follower in followers_of_friends.followers(count=10):
        follower_of_followers.append(follower.screen_name)
print(follower_of_followers)

In [ ]:
# Bonus Exercise 3
follower_list = []
follower_but_not_follower_of_follower = []
user = api.get_user('Nathananael_J')

for follower in user.followers(count=2):
    follower_list.append(follower.screen_name)
    
for name in follower_list:
    followers_of_friends = api.get_user(name)
    for my_follower in followers_of_friends.followers(count=2):
        if my_follower not in follower_but_not_follower_of_followers:
            follower_but_not_follower_of_followers.append(my_follower.screen_name)
print(follower_but_not_follower_of_followers)